In [1]:
import torch
print(torch.cuda.is_available())


True


In [2]:
print(torch.version.cuda)


12.1


In [4]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [5]:
df = pd.read_csv("teste.csv")

In [6]:
df.head()

,hash,GetProcAddress,ExitProcess,WriteFile,GetLastError,CloseHandle,FreeLibrary,Sleep,GetStdHandle,MultiByteToWideChar,GetCurrentThreadId,FindClose,LeaveCriticalSection,EnterCriticalSection,VirtualAlloc,DeleteCriticalSection,WideCharToMultiByte,RegCloseKey,ReadFile,SetFilePointer,GetCurrentProcess,GetACP,UnhandledExceptionFilter,RaiseException,RtlUnwind,TlsGetValue,TlsSetValue,GetModuleHandleW,GetCPInfo,GetCurrentProcessId,VirtualFree,GetDC,GetModuleFileNameW,CreateFileW,SetEndOfFile,SetLastError,GetCommandLineW,DestroyWindow,QueryPerformanceCounter,VirtualQuery,ShowWindow,GetFileType,TerminateProcess,HeapFree,HeapAlloc,CreateThread,BeginPaint,EndPaint,GlobalAlloc,GetModuleHandleA,GetCommandLineA,GetWindowRect,DeleteObject,SetWindowPos,GetClientRect,SelectObject,LoadLibraryExW,GlobalLock,GlobalUnlock,GetSystemMetrics,IsWindow,GetSysColor,GetDeviceCaps,MulDiv,IsDebuggerPresent,InvalidateRect,SetTimer,LoadLibraryA,IsWindowVisible,HeapSize,GetStartupInfoW,GetFileSize,TranslateMessage,FillRect,VariantInit,SetForegroundWindow,SysFreeString,LoadResource,ReleaseDC,SizeofResource,GetSystemInfo,SetEvent,LocalFree,ScreenToClient,VirtualProtect,ResetEvent,GetTickCount,SetTextColor,SetBkMode,VariantClear,SetUnhandledExceptionFilter,CreateSolidBrush,LockResource,DeleteDC,GetStockObject,GetVersion,GetParent,SetFocus,GetSystemTimeAsFileTime,CreateCompatibleDC,RegOpenKeyExW,HeapReAlloc,LCMapStringW,RegQueryValueExW,UpdateWindow,CoCreateInstance,GetOEMCP,KillTimer,GetConsoleCP,GetWindow,GetEnvironmentStringsW,GetStringTypeW,GetProcessHeap,ClientToScreen,FreeEnvironmentStringsW,TlsAlloc,GetFocus,GetDesktopWindow,FlushFileBuffers,GetLocaleInfoW,TlsFree,GetDlgItem,WaitForSingleObject,InitializeCriticalSection,CreateWindowExW,SysAllocStringLen,SetStdHandle,IsValidCodePage,InitializeCriticalSectionAndSpinCount,LocalAlloc,GetConsoleMode,CoTaskMemFree,RedrawWindow,WriteConsoleW,CreateCompatibleBitmap,IsChild,OleInitialize,InterlockedIncrement,InterlockedDecrement,DispatchMessageW,GetCursorPos,BitBlt,DefWindowProcW,SendMessageW,IsProcessorFeaturePresent,SetWindowLongW,OleUninitialize,ReleaseCapture,SetCapture,LoadCursorW,CompareStringW,GetWindowThreadProcessId,CharNextW,FindNextFileW,GetModuleFileNameA,CreateEventW,GetFileAttributesW,RegSetValueExW,DecodePointer,CallWindowProcW,RegCreateKeyExW,EnumWindows,PeekMessageW,FindResourceW,GetDIBits,GetWindowLongW,SetWindowTextW,GetThreadLocale,MessageBoxW,IsValidLocale,RegDeleteValueW,RegDeleteKeyW,SetFilePointerEx,PostMessageW,EncodePointer,CoInitialize,FormatMessageW,GetWindowTextW,GetModuleHandleExW,SetErrorMode,GetUserDefaultLCID,GetObjectW,CoUninitialize,CreateFileA,GlobalFree,CoTaskMemAlloc,EnableWindow,PostQuitMessage,LoadLibraryExA,RegEnumKeyExW,GetClassNameW,DrawTextW,GetClassInfoExW,RegQueryInfoKeyW,CreateStreamOnHGlobal,RegisterWindowMessageW,RegOpenKeyExA,MessageBoxA,RegQueryValueExA,CreateAcceleratorTableW,lstrcmpW,SetCursor,CreateFontIndirectW,EnumSystemLocalesW,HeapDestroy,LoadBitmapW,SetBkColor,IsWindowEnabled,EnableMenuItem,CreateDirectoryW,InitializeSListHead,CreatePopupMenu,TrackPopupMenu,EqualRect,GetSystemMenu,DispatchMessageA,CharNextA,CreateBrushIndirect,ImageList_Create,ImageList_Destroy,PeekMessageA,UnregisterClassW,EnumChildWindows,FindFirstFileA,SendMessageA,GetMessageW,MapWindowPoints,lstrcmpiW,CloseClipboard,OutputDebugStringW,OpenClipboard,SetClipboardData,EmptyClipboard,GetLocaleInfoA,lstrlenA,FindFirstFileW,LoadLibraryW,FindFirstFileExW,ImageList_Add,VariantChangeType,CoGetClassObject,ShellExecuteW,VariantCopy,SysAllocString,GetLocalTime,MoveWindow,lstrlenW,RegisterClassExW,SysStringLen,ClosePrinter,SetFileAttributesW,AdjustWindowRectEx,GetKeyState,GetForegroundWindow,GetWindowTextLengthW,IsIconic,GetMenu,DestroyMenu,GetActiveWindow,GetEnvironmentVariableW,SetRect,LoadTypeLib,StretchBlt,GetSubMenu,GetFullPathNameW,SysReAllocStringLen,CreatePalette,LoadStringA,DestroyIcon,SaveDC,FreeResource,CreateDIBSection,GetSysColorBrush,LineTo,WindowFromPoint,GetMenuItemCount,MoveToEx,IsZo

In [8]:
list(df.columns)

['hash',
 'GetProcAddress',
 'ExitProcess',
 'WriteFile',
 'GetLastError',
 'CloseHandle',
 'FreeLibrary',
 'Sleep',
 'GetStdHandle',
 'MultiByteToWideChar',
 'GetCurrentThreadId',
 'FindClose',
 'LeaveCriticalSection',
 'EnterCriticalSection',
 'VirtualAlloc',
 'DeleteCriticalSection',
 'WideCharToMultiByte',
 'RegCloseKey',
 'ReadFile',
 'SetFilePointer',
 'GetCurrentProcess',
 'GetACP',
 'UnhandledExceptionFilter',
 'RaiseException',
 'RtlUnwind',
 'TlsGetValue',
 'TlsSetValue',
 'GetModuleHandleW',
 'GetCPInfo',
 'GetCurrentProcessId',
 'VirtualFree',
 'GetDC',
 'GetModuleFileNameW',
 'CreateFileW',
 'SetEndOfFile',
 'SetLastError',
 'GetCommandLineW',
 'DestroyWindow',
 'QueryPerformanceCounter',
 'VirtualQuery',
 'ShowWindow',
 'GetFileType',
 'TerminateProcess',
 'HeapFree',
 'HeapAlloc',
 'CreateThread',
 'BeginPaint',
 'EndPaint',
 'GlobalAlloc',
 'GetModuleHandleA',
 'GetCommandLineA',
 'GetWindowRect',
 'DeleteObject',
 'SetWindowPos',
 'GetClientRect',
 'SelectObject',
 'Lo

In [1]:
import os
import numpy as np
import cv2

# Caminho dos arquivos binários e onde as imagens processadas serão salvas
data_dir = './malevis_train_val_224x224/train'
processed_dir = './malevis_train_val_224x224/processed'

# Definir o tamanho da imagem original e redimensionada
IMG_SIZE_ORIG = (224, 224)  # Tamanho da imagem original após conversão do binário
IMG_SIZE_RESIZED = (128, 128)  # Tamanho da imagem redimensionada para vetorizar
VECTOR_SIZE = IMG_SIZE_RESIZED[0] * IMG_SIZE_RESIZED[1]  # Vetor 1x16384

# Função para garantir que o diretório exista
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Função para verificar se o arquivo é binário (baseado na extensão ou tipo de arquivo)
def is_binary_file(filename):
    return filename.lower().endswith(('.bin', '.exe', '.dll'))  # Adapte conforme suas necessidades

# Função para verificar se o arquivo é uma imagem
def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))

# Função para converter o binário em uma imagem de 224x224 em escala de cinza
def bin_to_image(bin_file, size=(224, 224)):
    with open(bin_file, 'rb') as f:
        byte_array = np.frombuffer(f.read(), dtype=np.uint8)
    
    # Ajustar o tamanho da imagem, lidando com arquivos pequenos e grandes
    img_size = size[0] * size[1]
    
    if len(byte_array) < img_size:
        # Preencher com zeros se o arquivo for muito pequeno
        byte_array = np.pad(byte_array, (0, img_size - len(byte_array)), mode='constant')
    else:
        # Truncar se o arquivo for muito grande
        byte_array = byte_array[:img_size]
    
    # Converter os bytes para uma imagem 2D (escala de cinza)
    img = byte_array.reshape(size)
    return img

# Função para processar e binarizar a imagem
def process_image(img, size_resized=(128, 128)):
    # Redimensionar para 128x128
    img_resized = cv2.resize(img, size_resized)
    
    # Vetorizar a imagem (1x16384)
    img_vector = img_resized.flatten()
    
    # Binarização: valores < 128 tornam-se 0, valores >= 128 tornam-se 1
    img_binary = np.where(img_vector < 128, 0, 1)
    
    return img_binary

# Função para salvar o vetor binarizado
def save_processed_vector(vector, save_path):
    np.save(save_path, vector)
    print(f"Vetor binarizado salvo em: {save_path}.npy")

# Função para processar arquivos e pastas
def process_and_save_images(data_dir, processed_dir):
    # Garantir que o diretório de destino exista
    ensure_dir(processed_dir)
    
    items = sorted(os.listdir(data_dir))  # Pode ser tanto pastas quanto arquivos
    
    for item in items:
        item_path = os.path.join(data_dir, item)
        processed_item_path = os.path.join(processed_dir, item)

        if os.path.isdir(item_path):  # Se for uma pasta
            ensure_dir(processed_item_path)  # Criar pasta no destino
            print(f"Processando imagens da pasta: {item}")
            
            # Processar cada arquivo dentro da pasta
            for file_name in os.listdir(item_path):
                file_path = os.path.join(item_path, file_name)
                
                if is_binary_file(file_name):
                    img = bin_to_image(file_path)
                    img_binary = process_image(img)
                    save_processed_vector(img_binary, os.path.join(processed_item_path, file_name))
                
                elif is_image_file(file_name):
                    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Carregar como escala de cinza
                    if img is not None:
                        img_binary = process_image(img)
                        save_processed_vector(img_binary, os.path.join(processed_item_path, file_name))
                    else:
                        print(f"Falha ao processar a imagem {file_name}.")
                
                else:
                    print(f"{file_name} não é um arquivo válido e foi ignorado.")
        
        elif is_binary_file(item):  # Se for um arquivo binário na raiz
            print(f"Processando arquivo binário na raiz: {item}")
            img = bin_to_image(item_path)
            img_binary = process_image(img)
            save_processed_vector(img_binary, processed_item_path)
        
        elif is_image_file(item):  # Se for uma imagem na raiz
            print(f"Processando arquivo de imagem na raiz: {item}")
            img = cv2.imread(item_path, cv2.IMREAD_GRAYSCALE)  # Carregar como escala de cinza
            if img is not None:
                img_binary = process_image(img)
                save_processed_vector(img_binary, processed_item_path)
            else:
                print(f"Falha ao processar a imagem {item}.")
        
        else:
            print(f"{item} não é um arquivo válido e foi ignorado.")

# Executar o pré-processamento e salvar os vetores binarizados
process_and_save_images(data_dir, processed_dir)

print("Todos os arquivos foram processados e salvos.")


Processando imagens da pasta: Adposhel
Vetor binarizado salvo em: ./malevis_train_val_224x224/processed/Adposhel/010e3d42f4a163706efb4a3f3c8e0f780d05621bresized_image.png.npy
Vetor binarizado salvo em: ./malevis_train_val_224x224/processed/Adposhel/01164710189f86ca25cb99cab3e060d9d417c787resized_image.png.npy
Vetor binarizado salvo em: ./malevis_train_val_224x224/processed/Adposhel/013381e81ff10e541fb0750e11dcc4a85b558b5dresized_image.png.npy
Vetor binarizado salvo em: ./malevis_train_val_224x224/processed/Adposhel/01ec973597a667f17a2e9a1142e498f3176e7578resized_image.png.npy
Vetor binarizado salvo em: ./malevis_train_val_224x224/processed/Adposhel/04028a976dfcfb040a6238dc37bef7ab85a6f606resized_image.png.npy
Vetor binarizado salvo em: ./malevis_train_val_224x224/processed/Adposhel/052ccc067b4c26eb42412b80f8364980c0955469resized_image.png.npy
Vetor binarizado salvo em: ./malevis_train_val_224x224/processed/Adposhel/05a2bc7c98b617940136599050042ded65da846cresized_image.png.npy
Vetor bin

In [3]:
import pandas as pd
teste = pd.read_csv("dynamic_api_call_sequence_per_malware_100_0_306.csv")

In [4]:
teste.head()

,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,malware
0,071e8c3f8922e186e57548cd4c703a5d,112,274,158,215,274,158,215,298,76,...,71,297,135,171,215,35,208,56,71,1
1,33f8e6d08a6aae939f25a8e0d63dd523,82,208,187,208,172,117,172,117,172,...,81,240,117,71,297,135,171,215,35,1
2,b68abd064e975e1c6d5f25e748663076,16,110,240,117,240,117,240,117,240,...,65,112,123,65,112,123,65,113,112,1
3,72049be7bd30ea61297ea624ae198067,82,208,187,208,172,117,172,117,172,...,208,302,208,302,187,208,302,228,302,1
4,c9b3700a77facf29172f32df6bc77f48,82,240,117,240,117,240,117,240,117,...,209,260,40,209,260,141,260,141,260,1
